In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, utils, models, callbacks as cb
from keras.utils.vis_utils import plot_model

In [2]:
image_size = (671, 471)
batch_size = 10

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "training",
    validation_split=0.2,
    subset="training",
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    class_names=["forged", "genuine"],
    label_mode="binary",
    seed = 3301
)

train_ds = train_ds.prefetch(buffer_size=10)

Found 35 files belonging to 2 classes.
Using 28 files for training.


In [3]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(15, 25))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(int(labels[i]))
#         plt.axis("off")


In [4]:

# def make_model(input_shape, num_classes):
#     inputs = keras.Input(shape=input_shape)
#     # Image augmentation block
#     data_augmentation = keras.Sequential(
#         [
#             layers.RandomFlip("horizontal"),
#             layers.RandomRotation(0.1),
#         ]
#     )
#     x = data_augmentation(inputs)

#        # Entry block
#     x = layers.Rescaling(1.0 / 255)(x)
#     x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)

#     x = layers.Conv2D(64, 3, padding="same")(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)

#     previous_block_activation = x  # Set aside residual

#     for size in [128, 256, 512, 728]:
#         x = layers.Activation("relu")(x)
#         x = layers.SeparableConv2D(size, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.Activation("relu")(x)
#         x = layers.SeparableConv2D(size, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

#         # Project residual
#         residual = layers.Conv2D(size, 1, strides=2, padding="same")(
#             previous_block_activation
#         )
#         x = layers.add([x, residual])  # Add back residual
#         previous_block_activation = x  # Set aside next residual

#     x = layers.SeparableConv2D(1024, 3, padding="same")(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)

#     x = layers.GlobalAveragePooling2D()(x)
#     if num_classes == 2:
#         activation = "sigmoid"
#         units = 1
#     else:
#         activation = "softmax"
#         units = num_classes

#     x = layers.Dropout(0.5)(x)
#     outputs = layers.Dense(units, activation=activation)(x)
#     return keras.Model(inputs, outputs)

# model = make_model(input_shape=image_size + (3,), num_classes=2)
# # keras.utils.plot_model(model, show_shapes=True)

In [5]:
def make_model(input_shape, num_classes):
    return keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(0.2),
            layers.Dense(1, activation="sigmoid"),
        ]
    )


model = make_model(input_shape=image_size + (3,), num_classes=2)


In [6]:
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint("saves/{epoch}.h5"),
    keras.callbacks.TensorBoard(log_dir="./logs"),
]

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["binary_accuracy"],
)

model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    # validation_data=val_ds,
)


Epoch 1/25
3/3 [==============================] - 7s 539ms/step - loss: 0.4862 - binary_accuracy: 0.5357
Epoch 2/25
3/3 [==============================] - 0s 128ms/step - loss: 0.1146 - binary_accuracy: 1.0000
Epoch 3/25
3/3 [==============================] - 0s 134ms/step - loss: 0.1698 - binary_accuracy: 0.9643
Epoch 4/25
3/3 [==============================] - 0s 119ms/step - loss: 0.0382 - binary_accuracy: 0.9643
Epoch 5/25
3/3 [==============================] - 0s 123ms/step - loss: 0.0163 - binary_accuracy: 1.0000
Epoch 6/25
3/3 [==============================] - 0s 154ms/step - loss: 0.0040 - binary_accuracy: 1.0000
Epoch 7/25
3/3 [==============================] - 0s 122ms/step - loss: 1.5740e-04 - binary_accuracy: 1.0000
Epoch 8/25
3/3 [==============================] - 0s 114ms/step - loss: 5.3534e-06 - binary_accuracy: 1.0000
Epoch 9/25
3/3 [==============================] - 0s 99ms/step - loss: 4.7379e-04 - binary_accuracy: 1.0000
Epoch 10/25
3/3 [===========================

In [7]:
model.save("thing100.h5")

In [8]:
# ##### plotting the model

# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

# # tf.keras.utils.plot_model(
# #     model,
# #     to_file='model.png',
# #     show_shapes=False,
# #     show_dtype=False,
# #     show_layer_names=True,
# #     rankdir='TB',
# #     expand_nested=False,
# #     dpi=96,
# #     layer_range=None,
# #     show_layer_activations=False
# # )